In [2]:
import pandas as pd
import re
import io
import os
import glob
import time
from unicodedata import normalize
from sqlalchemy import create_engine
import psycopg2 as ps
from sqlalchemy import text

#database information to connect
host = 'ds4ateam77.cxdzzcrtbiby.us-east-2.rds.amazonaws.com'
port = 5432
user = 'postgres'
password = '12345678'
database = 'teate'
pd.set_option('display.max_columns', None)
def runQuery(sql):
    engine=create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}',max_overflow=30)
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())


In [6]:
start_time = time.time()
SQL_Query = runQuery("""

select * from fact_orders_clean
limit 100
""") 
print(str(len(SQL_Query))+' rows')
print("--- %s seconds ---" % (time.time() - start_time))
SQL_Query.head(10)


100 rows
--- 2.498607873916626 seconds ---


,index,pedido,fechapedido,tienda,nombretienda,direcciontienda,fabricante,nombrefabricante,material,nombrematerial,cantidaddepedido,um,valorunitariopedido,valortotalpedido,entrega,factura,zona,nombrezonacomercial,ruta,nombreruta,comuna,barrio,poblacion,row_number,month,year,day
0,242780,1000020182,2019-01-02,20000541,Tienda Mixta Nico,CR 7 C BIS # 81 - 136,5000044,INDUSTRIAS PATOJITO S.A.S,412,Desmanchador Ropa Color 450 ml,25,UN,1650,41250,3000025628,25259,None,None,V,Viernes,COMUNA 7,ALFONZO LOPEZ 1 ETAPA,Santiago de cali,1,1,2019,2
1,242778,1000020182,2019-01-02,20000541,Tienda Mixta Nico,CR 7 C BIS # 81 - 136,5000021,ARROZ FLORHUILA S.A,55,Florhuila Arroba 25 unid. x 500 g,1,ARR,29200,29200,3000025629,25369,None,None,V,Viernes,COMUNA 7,ALFONZO LOPEZ 1 ETAPA,Santiago de cali,1,1,2019,2
2,242779,1000020182,2019-01-02,20000541,Tienda Mixta Nico,CR 7 C BIS # 81 - 136,5000020,ORGANIZACION ROA S.A,57,Roa Arroba 25 unid. x 500 g,1,ARR,29200,29200,3000025629,25369,None,None,V,Viernes,COMUNA 7,ALFONZO LOPEZ 1 ETAPA,Santiago de cali,1,1,2019,2
3,242781,1000020183,2019-01-02,20001936,Tienda Mixta Claudia,CL 81 7 E BIS 38,5000026,COMESTIBLES ALDOR S.A.S,483,Play by trolli surtido,1,CJ,12300,12300,3000025630,25370,None,None,V,Viernes,COMUNA 7,Alfonzo lopez 2 etapa,Santiago de cali,1,1,2019,2
4,242782,1000020183,2019-01-02,20001936,Tienda Mixta Claudia,CL 81 7 E BIS 38,5000020,ORGANIZACION ROA S.A,57,Roa Arroba 25 unid. x 500 g,1,ARR,29200,29200,3000025630,25370,None,None,V,Viernes,COMUNA 7,Alfonzo lopez 2 etapa,Santiago de cali,1,1,2019,2
5,242785,1000020184,2019-01-02,20001348,Tienda Variedades Nahiara,CL 108 # 26 S - 31,5000042,BRINSA S.A.,315,Blq blancox poder natural x 500ml,10,UN,800,8000,3000025632,25260,None,None,J,Jueves,COMUNA 14,Manuela Beltran,Santiago de cali,1,1,2019,2
6,242786,1000020184,2019-01-02,20001348,Tienda Variedades Nahiara,CL 108 # 26 S - 31,5000051,DETERGENTES LTDA.,498,Detetergente top basico Floral 250gr,9,UN,1550,13950,3000025632,25260,None,None,J,Jueves,COMUNA 14,Manuela Beltran,Santiago de cali,1,1,2019,2
7,242783,1000020184,2019-01-02,20001348,Tienda Variedades Nahiara,CL 108 # 26 S - 31,5000008,HARINERA DEL VALLE S.A,88,Aceite Oleocali x 250,6,UN,1600,9600,None,None,None,None,J,Jueves,COMUNA 14,Manuela Beltran,Santiago de cali,1,1,2019,2
8,242784,1000020184,2019-01-02,20001348,Tienda Variedades Nahiara,CL 108 # 26 S - 31,5000008,HARINERA DEL VALLE S.A,89,Aceite Oleocali x 500,4,UN,2950,11800,None,None,None,None,J,Jueves,COMUNA 14,Manuela Beltran,Santiago de cali,1,1,2019,2
9,242787,1000020185,2019-01-02,20000638,Tienda Edwin,CR 24 B # 48 - 04,5000042,BRINSA S.A.,315,Blq blancox poder natural x 500ml,12,UN,800,9600,3000025633,25261,None,None,M,Martes,COMUNA 12,Fenalco Kenedy,Santiago de cali,1,1,2019,2


In [5]:
start_time = time.time()
SQL_Query = runQuery("""

DROP TABLE fact_orders_clean;
CREATE TABLE IF NOT EXISTS fact_orders_clean as (
with base as (
    select 
        index, pedido, fechapedido::DATE,
        tienda, nombretienda,
       direccintienda as direcciontienda, fabricante, nombrefabricante,
       material, nombrematerial, cantidaddepedido::INT, um,
       valorunitariopedido::INT,  valortotalpedido::INT,
       entrega, factura, zona, nombrezonacomercial, ruta,
       nombreruta, comuna, barrio, poblacin as poblacion, 
        row_number() over(partition by pedido, material  order by valorunitariopedido desc) as row_number,
        EXTRACT(MONTH FROM fechapedido::DATE)::INT AS month,
        EXTRACT(YEAR FROM fechapedido::DATE)::INT AS year,
        EXTRACT(DAY FROM fechapedido::DATE)::INT AS day
    from fact_orders_s
),
base2 as(
    select * from base
    where row_number =1)
    
select *
from base2)

""") 
print(str(len(SQL_Query))+' rows')
print("--- %s seconds ---" % (time.time() - start_time))
SQL_Query.head(10)


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [66]:
start_time = time.time()
SQL_Query = runQuery("""

select 
      year
    , month
    , count(distinct pedido) Oders
    , sum(valortotalpedido) 

from  fact_orders_clean
    group by 1,2
    order by 1
""") 
print(str(len(SQL_Query))+' rows')
print("--- %s seconds ---" % (time.time() - start_time))
SQL_Query.head(50)


21 rows
--- 5.340271234512329 seconds ---


,year,month,oders,sum
0,2019,1,3094,176229577
1,2019,2,4349,254878461
2,2019,3,5229,292848722
3,2019,4,5196,318994075
4,2019,5,6374,395694820
5,2019,6,4623,272913350
6,2019,7,4996,294477300
7,2019,8,6188,421424847
8,2019,9,6826,487792740
9,2019,10,7986,519116086
